In [18]:
import numpy as np
import pandas as pd
import networkx as nx
import torch
from torch_geometric.utils import coalesce, remove_self_loops
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
from scipy import io

In [40]:
df = pd.read_csv('R5.2_sessions.csv')
df

,user_code,pc_code
0,927,32
1,927,466
2,792,1848
3,927,1088
4,1128,536
...,...,...
995077,283,1715
995078,1931,383
995079,460,859
995080,1144,962


In [41]:
edge_list = [(x, y) for x, y in zip(df['user_code'], df['pc_code'])]
edge_list

[(927, 32),
 (927, 466),
 (792, 1848),
 (927, 1088),
 (1128, 536),
 (927, 1439),
 (812, 1585),
 (736, 1674),
 (1455, 1440),
 (1513, 5),
 (447, 1479),
 (927, 438),
 (1465, 146),
 (1541, 594),
 (833, 139),
 (1578, 590),
 (1170, 236),
 (1188, 1313),
 (186, 513),
 (85, 1061),
 (816, 1468),
 (1874, 1138),
 (65, 1969),
 (590, 1557),
 (531, 1549),
 (261, 936),
 (1873, 1108),
 (1021, 1470),
 (1799, 1041),
 (1357, 127),
 (1693, 418),
 (45, 1807),
 (893, 843),
 (891, 566),
 (1321, 1907),
 (531, 1447),
 (258, 1744),
 (506, 316),
 (590, 1264),
 (65, 240),
 (586, 371),
 (997, 1412),
 (1825, 2001),
 (1083, 721),
 (717, 1256),
 (1059, 525),
 (1203, 1604),
 (1538, 1283),
 (1674, 685),
 (1750, 1408),
 (602, 1194),
 (847, 886),
 (1983, 142),
 (784, 443),
 (1831, 541),
 (796, 1513),
 (1422, 1500),
 (827, 1285),
 (943, 681),
 (1792, 294),
 (30, 393),
 (651, 1613),
 (1945, 356),
 (1956, 1051),
 (209, 1057),
 (815, 1654),
 (1214, 1321),
 (1706, 838),
 (955, 311),
 (1113, 816),
 (1467, 1962),
 (542, 1606),
 

In [42]:
G = nx.from_edgelist(edgelist=edge_list)
nodes_degree = list(dict(G.degree()).values())
A = nx.adjacency_matrix(G)
A = A.tocoo()

In [43]:
A

<COOrdinate sparse array of dtype 'int64'
	with 130183 stored elements and shape (2004, 2004)>

In [44]:
io.savemat('./data/R52_SESSION.mat', {'net': A})

In [8]:
from scipy.io import loadmat

In [45]:
x = loadmat('data/R52_SESSION.mat')

In [46]:
x

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Feb 27 16:08:22 2025',
 '__version__': '1.0',
 '__globals__': [],
 'net': <COOrdinate sparse matrix of dtype 'int64'
 	with 130183 stored elements and shape (2004, 2004)>}

In [23]:
x['net'].col

array([   0,    0,    0, ..., 1002, 1002, 1002], dtype=int32)

In [12]:
from src.utils import load_unsplitted_data

In [47]:
class data:
    def __init__(self, data_name):
        self.data_name = data_name

args = data('R52_SESSION')

In [25]:

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch
import argparse
import numpy as np
import math
from torch_geometric.utils import to_undirected, from_scipy_sparse_matrix,dense_to_sparse,is_undirected
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
import sys
import os.path
import pickle as pkl
import networkx as nx
import scipy.sparse as sp

# cur_dir = os.path.dirname(os.path.realpath(__file__))
# par_dir = os.path.abspath(os.path.join(os.path.dirname(__file__),".."))
# sys.path.append('%s/software/' % par_dir)

def load_unsplitted_data(args):
    # read .mat format files
    data_dir = os.path.join('data/{}.mat'.format(args.data_name))
    print('Load data from: '+ data_dir)
    import scipy.io as sio
    net = sio.loadmat(data_dir)
    print(net)
    edge_index,_ = from_scipy_sparse_matrix(net['net'])
    data = Data(edge_index=edge_index)
    if is_undirected(data.edge_index) == False: #in case the dataset is directed
        data.edge_index = to_undirected(data.edge_index)
    data.num_nodes = torch.max(data.edge_index)+1
    return data

In [48]:
load_unsplitted_data(args)

Load data from: data/R52_SESSION.mat
{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Feb 27 16:08:22 2025', '__version__': '1.0', '__globals__': [], 'net': <COOrdinate sparse matrix of dtype 'int64'
	with 130183 stored elements and shape (2004, 2004)>}


Data(edge_index=[2, 130183], num_nodes=2004)